In [1]:
### This script computes inter-protein contacts throughout an MD trajectory

### Inputs: TPR file, XTC files, protein_indeces.txt files, Proteome Class code, Threshold radius
### Output: List of Contacts

In [ ]:
import sys
import os
sys.path.append('../ProteomeClass/') 
from proteome import *
import re
from scipy.stats import gaussian_kde

#### USER modifications ####################################
directory    = '../../Data.Availability/Upload/TRJ_Psychro_sys1/System1/'
indices_file = directory + 'Files/protein_indeces.txt'
TPR_file     = directory + '/Folded/300K/sys_prot.tpr'
XTC_file     = directory + '/Folded/%sK/traj_prot.xtc'
temps        = [300]

output_dir   = 'Contacts_PA_test'
nr_processes = 10
radius_cnt   = 3    # Angstroms
skip         = 5 # skip rate frames
###########################################################


prot1 = Proteome(tpr_file     = TPR_file, 
                 indices_path = indices_file,
                 outputdir    = output_dir, 
                 base         = 'P_Arcticus',
                 plot         = True,
                 selection    = "protein"
                )

output_directory = os.path.join('./',output_dir)  # Define the new directory path
os.makedirs(output_directory, exist_ok=True)  # Create the directory if it doesn't exist

for T in temps:
    print('Processing Temperature ', T, ' K')
    output_directory_curr = os.path.join(output_directory, '%sK'%T)  # Define the new directory path
    os.makedirs(output_directory_curr, exist_ok=True)  # Create the directory if it doesn't exist
    nums = '0'

    prot1.add_trajectory(XTC_file%T)
    prot1.add_wrapped_trajectory(XTC_file%T)
    com = prot1.center_of_masses()
    prot1.write_com_to_xtc(filename = 'COM_trajectory')
            
    Contacts, List,_ = prot1.contacts(type_cont='allres', radius_contact=radius_cnt, skip=skip, processes = nr_processes, 
                                      create_trajectory=False, show_wrap=True, output_file="Trajectory.pdb")
